<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [5]:
## Task 1: Mark all launch sites on a map

from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

# Create a folium map centered around the average coordinates
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long']]
launch_map = folium.Map(location=[spacex_df["Lat"].mean(), spacex_df["Long"].mean()], zoom_start=5)

# display
display(launch_map)

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Loop through each launch site and add it to the map
for index, row in launch_sites_df.iterrows():
    coordinate = [row["Lat"], row["Long"]]

    # Add a circle at each launch site
    folium.Circle(
        location=coordinate,
        radius=1000,  # 2km radius for visibility
        color="orange",
        fill=True,
        fill_color="orange",
        fill_opacity=0.3
    ).add_to(site_map)

    # Add a marker with bold text label
    folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 14px; color: orange; font-weight: bold;">{row["Launch Site"]}</div>',
        ),
    ).add_to(site_map)

# Display the final map
display(site_map)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [11]:
# Task 2: Mark the success/failed launches for each site on the map

import folium
from folium.features import DivIcon

# Define a function to set marker color based on launch success/failure
def get_marker_color(launch_status):
    return "green" if launch_status == 1 else "red"

# Start location: NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)  # Adjust zoom for visibility

# Loop through each launch in the dataset
for index, row in spacex_df.iterrows():
    coordinate = [row["Lat"], row["Long"]]
    marker_color = get_marker_color(row["class"])  # Determine color (Success = Green, Failure = Red)

    # Add a marker for each launch with color-coded status
    folium.Circle(
        location=coordinate,
        radius=150,  # Smaller radius for each launch
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.5,
    ).add_to(site_map)

    # Add text labels with success/failure info
    folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(10, 10),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 10px; color: {marker_color};"><b>{"Success" if row["class"] == 1 else "Failure"}</b></div>',
        ),
    ).add_to(site_map)

# Display the final map
display(site_map)



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [13]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [14]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Define a function to assign colors based on class value
def assign_marker_color(launch_status):
    return "green" if launch_status == 1 else "red"

# Create a new column 'marker_color' in spacex_df
spacex_df["marker_color"] = spacex_df["class"].apply(assign_marker_color)

# Display the first few rows to verify
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [15]:
# Add marker_cluster to current site_map
import folium
from folium.plugins import MarkerCluster
from folium import Icon

# Define NASA start location
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Define marker color based on launch success (1 = green, 0 = red)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Add markers for each launch event
for index, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=Icon(color='white', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}"
    ).add_to(marker_cluster)

# Show map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [16]:
# TASK 3: Calculate the distances between a launch site to its proximities
from math import radians, cos, sin, sqrt, atan2

# Function to calculate the distance between two coordinates
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance


In [17]:
# Example proximity locations (lat, lon)
proximity_locations = {
    "Nearest City": (28.3922, -80.6077),  # Example: Cape Canaveral
    "Nearest Highway": (28.5721, -80.6480),  # Example: Highway near Kennedy Space Center
    "Nearest Railway": (28.5721, -80.5853),  # Example: Railway near Kennedy Space Center
    "Coastline": (28.3922, -80.6000)  # Approximate coastline location
}


In [18]:
# Create a new DataFrame column to store distances
for site_index, site in launch_sites_df.iterrows():
    site_lat, site_lon = site['Lat'], site['Long']
    
    print(f"\nDistances from {site['Launch Site']}:")
    for place, (lat, lon) in proximity_locations.items():
        distance = haversine(site_lat, site_lon, lat, lon)
        print(f"  - {place}: {distance:.2f} km")



Distances from CCAFS LC-40:
  - Nearest City: 19.15 km
  - Nearest Highway: 6.98 km
  - Nearest Railway: 1.34 km
  - Coastline: 19.04 km

Distances from CCAFS SLC-40:
  - Nearest City: 19.25 km
  - Nearest Highway: 7.02 km
  - Nearest Railway: 1.29 km
  - Coastline: 19.15 km

Distances from KSC LC-39A:
  - Nearest City: 20.49 km
  - Nearest Highway: 0.17 km
  - Nearest Railway: 6.02 km
  - Coastline: 20.65 km

Distances from VAFB SLC-4E:
  - Nearest City: 3829.73 km
  - Nearest Highway: 3819.00 km
  - Nearest Railway: 3824.73 km
  - Coastline: 3830.43 km


In [19]:
for place, (lat, lon) in proximity_locations.items():
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='blue', icon="info-sign"),
        popup=f"{place}"
    ).add_to(site_map)


In [20]:
print(launch_sites_df.head())  # Check if the DataFrame has launch sites


    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610745


In [21]:
# Example Test: Distance between two known points
test_distance = haversine(28.5721, -80.6480, 28.3922, -80.6077)  
print(f"Test Distance: {test_distance:.2f} km")


Test Distance: 20.39 km


In [22]:
site_map.save("map.html")  # Save the map to an HTML file


In [23]:
from folium.plugins import MarkerCluster

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        icon=folium.Icon(color='green' if row["class"] == 1 else 'red'),
        popup=row["Launch Site"]
    ).add_to(marker_cluster)

# Display the map
site_map


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [24]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [25]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [26]:
# find coordinate of the closet coastline

from folium.plugins import MousePosition

# Function to display latitude and longitude on the map
mouse_position = MousePosition(position='topright', separator=' | ', empty_string='No coordinates',
                               lat_first=True, num_digits=5, prefix='Lat | Lon:')
site_map.add_child(mouse_position)

site_map  # Display map

# e.g.,: Lat: 28.56367  Lon: -80.57163

from math import radians, cos, sin, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    R = 6371  # Radius of the Earth in km
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c  # Distance in km

# Example launch site (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Closest coastline point
coastline_lat = 28.56367
coastline_lon = -80.57163

# Compute distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance from launch site to coastline: {distance_coastline:.2f} km")

# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Add a marker for the coastline point
folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(site_map)

site_map  # Display map


Distance from launch site to coastline: 0.51 km


In [30]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )


import folium
from folium.features import DivIcon


# Create a marker for the closest coastline point
coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="blue", icon="info-sign")
)

# Add the marker to the site map
site_map.add_child(coastline_marker)

# Create a distance label marker
distance_marker = folium.Marker(
    [ (launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2 ],  # Midpoint between the two locations
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the distance marker to the site map
site_map.add_child(distance_marker)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [31]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)

import folium
from folium.features import DivIcon

# Define coordinates for launch site and closest coastline
launch_site_coord = [launch_site_lat, launch_site_lon]
coastline_coord = [coastline_lat, coastline_lon]

# Create a PolyLine object
lines = folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],  # List of coordinates
    weight=2,  # Line thickness
    color="blue",  # Line color
    opacity=0.7
)

# Add PolyLine to site_map
site_map.add_child(lines)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [42]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [33]:
import folium
from folium.plugins import MousePosition
from folium.features import DivIcon
from geopy.distance import geodesic

# Define NASA Launch site coordinates
launch_site_coord = [28.5721, -80.6480]  # Example coordinate for a launch site

# Example closest locations (Find actual ones using Google Maps or GIS tools)
closest_city_coord = [28.6139, -80.5945]      # Example closest city
closest_railway_coord = [28.6000, -80.5833]   # Example closest railway
closest_highway_coord = [28.6090, -80.6046]   # Example closest highway

# Function to calculate distance between two points (Haversine formula)
from geopy.distance import geodesic

def calculate_distance(coord1, coord2):
    return round(geodesic(coord1, coord2).km, 2)  # Distance in KM

# Create a folium map
site_map = folium.Map(location=[28.5733, -80.6469], zoom_start=10)

# Add MousePosition plugin to get lat/lon coordinates on hover
MousePosition().add_to(site_map)

def add_distance_marker(map_object, launch_coord, target_coord, label, color):
    """
    Adds a marker and line between launch site and another location.
    """
    distance = geodesic(launch_coord, target_coord).km  # Compute distance

    # Add marker
    folium.Marker(
        target_coord,
        icon=folium.DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{label}: {round(distance,2)} KM</b></div>'
        )
    ).add_to(map_object)

    # Add a PolyLine between launch site and target
    folium.PolyLine(
        locations=[launch_site_coord, target_coord], weight=2, color="blue"
    ).add_to(map_object)

# Adding markers for each closest point
add_distance_marker(site_map, [28.5721, -80.5853], [28.3922, -80.57163], "Coastline", "red")
add_distance_marker(site_map, launch_site_coord, closest_city_coord, "City", "blue")
add_distance_marker(site_map, launch_site_coord, closest_railway_coord, "Railway", "green")
add_distance_marker(site_map, launch_site_coord, closest_highway_coord, "Highway", "purple")

# Show the map
site_map


<class 'ModuleNotFoundError'>: No module named 'geopy'

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
